<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

In [1]:
from eoxhub import check_compatibility
check_compatibility("user-2023.03-02", dependencies=[])

# Visualizing data with xcube viewer in EuroDataCube
### Brought to you by Brockmann Consult GmbH
This Notebook demonstrates how to use the features of the xcube JupyterLab integration.
The Notebook demonstrates three scenarios how xcube Viewer is utilized in JupyterLab.
In particular, we open xcube Viewer for any `xarray.Dataset` instances 

1. opened or otherwise created in this Notebook (in-memory datasets);
2. persisted in this Notebooks's workspace or from other sources (saved datasets).
3. open a dataset served by an external xcube server and published in a xcube viewer.

For this to work in EuroDataCube, the following requirements must be satisfied:

* you must have an active EuroDataCube EOxHub Workspace
* you must have an active xcube Viewer subscription

In [2]:
import numpy as np
import xarray as xr

from xcube.webapi.viewer import Viewer
from xcube.core.store import new_data_store
from xcube.core.select import select_subset

We create some datasets so we have something to show. We use the xcube datastore framework here to open the dataset, but it could also be opened by other means, e.g., `xr.open_dataset()`, provided it has variables with dimensions ["time", "y", "x"] or ["y", "x"]. 

If you have your own bucket, which you wish to access and the access credentials are not saved as environment variables, you can add them to the parameters of `new_data_store`:
```
store = new_data_store("s3",
                       root="PATH_TO_YOUR_S3_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key="YOUR_ACCESS_KEY_ID", 
                                            secret="YOUR_ACCESS_KEY_SECRET")
                           ) 
```

In [3]:
store = new_data_store("s3", 
                       root="xcube-dcfs/edc-xc-viewer-data",
                       storage_options={"anon": True})
store.list_data_ids()

['Aegean.zarr',
 'Aegean_2.zarr',
 'Bodensee_2.zarr',
 'C3S_ERA-5_t2m.zarr',
 'CCI-CHL.zarr',
 'CMEMS_OC_CHL.zarr',
 'CMEMS_OC_CHL_2.zarr',
 'GHSL_2018.zarr',
 'HH_CityCube_RGB.zarr',
 'NorthSea_2021-04_08.zarr']

In [4]:
dataset = store.open_data('HH_CityCube_RGB.zarr')

In [5]:
dataset

<xarray.Dataset>
Dimensions:    (time: 54, lat: 1536, lon: 2560, bnds: 2)
Coordinates:
  * lat        (lat) float64 53.85 53.85 53.85 53.85 ... 53.39 53.39 53.39 53.39
  * lon        (lon) float64 9.7 9.7 9.701 9.701 ... 10.47 10.47 10.47 10.47
  * time       (time) datetime64[ns] 2021-03-03 2021-03-07 ... 2021-10-01
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(54, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    B02        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B03        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B04        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B05        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B06        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B08        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    CLD        (time, lat, lon) uint8 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    MCI        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    NDVI       (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    SCL        (time, lat, lon) uint8 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.7
    date_created:               2021-10-02T17:07:09.898152
    geospatial_bounds:          POLYGON((9.700000000380166 53.39000000038017,...
    geospatial_bounds_crs:      CRS84
    geospatial_lat_max:         53.85471028749284
    geospatial_lat_min:         53.39000000038017
    ...                         ...
    processing_level:           L2A
    time_coverage_duration:     P216DT0H0M0S
    time_coverage_end:          2021-10-03T00:00:00+00:00
    time_coverage_resolution:   P4DT0H0M0S
    time_coverage_start:        2021-03-01T00:00:00+00:00
    title:                      Hamburg City Cube 2021

In [6]:
dataset_subset = select_subset(
    dataset,
    time_range=["2021-08-01 00:00:00", 
                "2021-08-31 23:59:59"],
    var_names=["B04", "B03", "B02", "NDVI"]
)
dataset_subset.attrs["title"] = "Hamburg NDVI and RGB Subset"
dataset_subset

<xarray.Dataset>
Dimensions:    (time: 8, lat: 1536, lon: 2560, bnds: 2)
Coordinates:
  * lat        (lat) float64 53.85 53.85 53.85 53.85 ... 53.39 53.39 53.39 53.39
  * lon        (lon) float64 9.7 9.7 9.701 9.701 ... 10.47 10.47 10.47 10.47
  * time       (time) datetime64[ns] 2021-08-02 2021-08-06 ... 2021-08-30
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(8, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    B02        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B03        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    B04        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
    NDVI       (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.7
    date_created:               2021-10-02T17:07:09.898152
    geospatial_bounds:          POLYGON((9.700000000380166 53.39000000038017,...
    geospatial_bounds_crs:      CRS84
    geospatial_lat_max:         53.85471028749284
    geospatial_lat_min:         53.39000000038017
    ...                         ...
    processing_level:           L2A
    time_coverage_duration:     P216DT0H0M0S
    time_coverage_end:          2021-10-03T00:00:00+00:00
    time_coverage_resolution:   P4DT0H0M0S
    time_coverage_start:        2021-03-01T00:00:00+00:00
    title:                      Hamburg NDVI and RGB Subset

---
**Scenario 1**: Open xcube Viewer for a dataset instances opened or otherwise created in this Notebook (in-memory datasets).   
**Documentation about the functionalities of xcube viewer is available here:** https://xcube.readthedocs.io/en/latest/viewer.html#functionality

In [7]:
viewer = Viewer()

In [8]:
viewer.add_dataset(dataset)
viewer.add_dataset(dataset_subset)

'219293b5-7131-4003-b49b-5d57f1300509'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [9]:
viewer.info()

Server: http://localhost:8000
Viewer: http://localhost:8000/viewer/?serverUrl=http://localhost:8000


You can also open xcube Viewer inlined here:

In [10]:
viewer.show()

To stop the server and viewer:

In [11]:
viewer.stop_server()

---
**Scenario 2**: Open xcube Viewer for a dataset instances persisted in this Notebooks's workspace or from other sources (saved datasets).

In [12]:
dataset_subset.to_zarr("hamburg-NDVI-RGB-subset.zarr", mode="w")

In [13]:
local_store = new_data_store("file", root="./.") # adjust the path if you store the subset somewhere else
local_store.list_data_ids()

['hamburg-NDVI-RGB-subset.zarr']

In [14]:
dataset = local_store.open_data('hamburg-NDVI-RGB-subset.zarr')

In [15]:
viewer = Viewer()

In [16]:
viewer.add_dataset(dataset)

'226ff15f-ca30-4943-afb0-a27b57459743'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [17]:
viewer.info()

Server: http://localhost:8001
Viewer: http://localhost:8001/viewer/?serverUrl=http://localhost:8001


You can also open xcube Viewer inlined here:

In [18]:
viewer.show()

In [19]:
viewer.stop_server()

---
**Scenario 3**: Use custom server configuration to start server and pass it to the viewer constructor. In this case, we have created a local file with the configuration and load it as a dictionary and pass it to the viewer. 

The custom configuration allows you to predefine your value ranges, the colormaps that should be used as well as which bands can be used to create an RGB image, then the RGB switch in the viewer will allow to show the RGB imgage. 

If you do not have a server-config.yaml file in your directory, please create one with the following content: 

```yaml
DataStores:
  - Identifier: edc
    StoreId: file
    StoreParams:
      root: "./." # adjust if you have stored the subset elsewhere
    Datasets:
      - Path: hamburg-NDVI-RGB-subset-1.zarr # adjust if you have named it differently.
        Style: default


Styles:
  - Identifier: default
    ColorMappings:
      NDVI:
        ColorBar: RdYlGn
        ValueRange: [-1., 1.]
      rgb:
        Red:
          Variable: B04
          ValueRange: [0., 0.25]
        Green:
          Variable: B03
          ValueRange: [0., 0.25]
        Blue:
          Variable: B02
          ValueRange: [0., 0.25]

```

To get details about the server configuration file, please checkout the documentation: https://xcube.readthedocs.io/en/latest/cli/xcube_serve.html     

In [20]:
from xcube.util.config import load_configs

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [21]:
viewer = Viewer(server_config=load_configs("server-config.yaml"))

ValueError: Cannot find configuration 'server-config.yaml'

You can also open xcube Viewer inlined here:

In [ ]:
viewer.show()

In [ ]:
viewer.stop_server()

Let's clean up and remove the test subset: 

In [ ]:
local_store.delete_data('hamburg-NDVI-RGB-subset.zarr')

---
**Scenario 4**: Open a dataset served from an external xcube server and published via xcube viewer.

Please head over to the eurodatacube xcube viewer: https://edc-viewer.brockmann-consult.de/
There please select the dataset called CMEMS Black Sea Chl. This dataset only has one variable, so you don't need to change the variable. 

Once you have selected the dataset, open the info panel on the right hand side of the xcube viewer. Then you will see a python icon in the panel next to the datasets title, please select it. 

Copy the displayed code snippet, and paste it into your jupyter notebook like done below:

In [ ]:
from xcube.core.store import new_data_store

store = new_data_store(
    "s3",
    root="datasets",  # can also use "pyramids" here
    storage_options={
        "anon": True,
        "client_kwargs": {
            "endpoint_url": "https://edc-api.brockmann-consult.de/api/s3"
        }
    }
)
# store.list_data_ids()
dataset = store.open_data(data_id="esdl~esdc-8d-0.25deg-1x720x1440-2.1.1.zarr")

In [ ]:
dataset

For the variable and the selected time stamp you will find a code snippet as well:

In [ ]:
var = dataset.air_temperature_2m.sel(time="2015-01-05 01:00:00", method="nearest")
var.plot.imshow(vmin=190, vmax=320, cmap="plasma")